# Tutorial 2: Tensor Basics

This notebook introduces tensors from 0D to 4D+, including creation, inspection, basic math, visualization, and a minimal store/retrieve flow with the Tensorus API (falls back to demo mode when the server is not available).

In [ ]:
# Lightweight install cell (safe to re-run)
import sys, subprocess, pkgutil
packages = ['numpy','torch','matplotlib','seaborn','requests']
for p in packages:
    if pkgutil.find_loader(p) is None:
        print(f'Installing {p} ...')
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', p])
print('✅ Dependencies ready')

In [ ]:
# Setup and helpers
import torch, numpy as np, requests, json
import matplotlib.pyplot as plt, seaborn as sns
sns.set_theme(style='whitegrid')
TENSORUS_API_URL = 'http://127.0.0.1:7860'
def server_ok():
    try:
        r = requests.get(f'{TENSORUS_API_URL}/health', timeout=2)
        return r.status_code == 200
    except Exception:
        return False
SERVER = server_ok()
print('📡 Tensorus:', '✅ Connected' if SERVER else '⚠️ Demo Mode')

## 0D (Scalar)

In [ ]:
temperature = torch.tensor(23.5)
price = torch.tensor(99.99)
print('Scalar:', temperature, '| dtype:', temperature.dtype, '| shape:', tuple(temperature.shape))
print('Fahrenheit:', (temperature * 9/5 + 32).item())

## 1D (Vector)

In [ ]:
stock = torch.tensor([100.0, 105.2, 98.7, 102.1, 110.5, 108.9, 115.3])
daily_returns = (stock[1:] - stock[:-1]) / stock[:-1] * 100
print('Vector shape:', tuple(stock.shape), 'norm:', torch.norm(stock).item())
print('Daily returns %:', daily_returns)
plt.figure(); plt.plot(stock.numpy()); plt.title('Stock Prices'); plt.show()

## 2D (Matrix)

In [ ]:
A = torch.randn(4, 4)
B = torch.randn(4, 4)
C = A @ B
print('A shape:', tuple(A.shape), 'det(A):', torch.det(A).item())
print('C shape:', tuple(C.shape))
plt.figure(); plt.imshow(C.detach().numpy(), cmap='viridis'); plt.colorbar(); plt.title('Matrix Product Heatmap'); plt.show()

## 3D (e.g., H×W×C image)

In [ ]:
img = torch.rand(64, 64, 3)
print('Image-like tensor:', tuple(img.shape))
plt.figure(); plt.imshow(img.numpy()); plt.title('Random Image'); plt.axis('off'); plt.show()

## 4D (Batch × H × W × C)

In [ ]:
batch = torch.rand(8, 64, 64, 3)
print('Batch shape:', tuple(batch.shape))
print('Per-image mean (first 3):', batch.view(8, -1).mean(dim=1)[:3])

## Store & Retrieve with Tensorus (fallback to demo)

In [ ]:
payload = {
  'tensor_data': img.tolist(),
  'metadata': {'name': 'sample_image', 'shape': list(img.shape)}
}
if SERVER:
    try:
        r = requests.post(f'{TENSORUS_API_URL}/api/v1/tensors', json=payload, timeout=5)
        tid = r.json().get('tensor_id', 'demo_tensor')
        rr = requests.get(f'{TENSORUS_API_URL}/api/v1/tensors/{tid}', timeout=5)
        print('Stored and retrieved tensor_id:', tid, '| status:', rr.status_code)
    except Exception as e:
        print('API error, demo mode:', e)
        print('Demo retrieved shape:', tuple(img.shape))
else:
    print('Demo stored tensor_id: demo_tensor | Demo retrieved shape:', tuple(img.shape))

## Takeaways
- Tensors generalize scalars, vectors, and matrices.
- Shape, dtype, and device are fundamental.
- Tensorus can persist tensors and metadata; notebooks work in connected or demo mode.